### Chicago Taxi Trip ETL pipeline project

In [1]:
# Import relevant modules

import io
import pandas as pd
import requests

In [2]:
# Load data using the google cloud storage url

url = 'https://storage.googleapis.com/chicago-taxi-trip-niktrack/taxi_trip_data.csv'
response = requests.get(url)

In [3]:
# Load data into csv dataframe

taxi_data = pd.read_csv(io.StringIO(response.text), sep=',')

In [4]:
# Read the dataset

taxi_data.head()

,taxi_trip_id,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,...,payment_type,company,dropoff_centroid_latitude,dropoff_centroid_longitude,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_communityID,dropoff_community_area,pickup_communityID,pickup_community_area
0,0,54d812a0b88f8f9707825261014b3563a0a60ace,f98ae5e71fdda8806710af321dce58002146886c013f41...,2022-01-01 00:00:00,2022-01-01 00:00:00,536,4.83,14.75,0.00,0.0,...,Cash,Globe Taxi,41.922761,-87.699155,41.874005,-87.663518,22,LOGAN SQUARE,28,NEAR WEST SIDE
1,1,7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef,8eca35a570101ad24c638f1f43eecce9d0cb7843e13a75...,2022-01-01 00:00:00,2022-01-01 00:15:00,897,2.07,9.75,0.00,0.0,...,Cash,Sun Taxi,41.878866,-87.625192,41.899602,-87.633308,32,LOOP,8,NEAR NORTH SIDE
2,2,f1a650ee419b4e52d766432e6f86eca3457bfb37,e2d8418fcdb061eee0a4318fba0a6a1200aaff0143feb0...,2022-01-01 00:00:00,2022-01-01 00:30:00,2200,2.48,9.36,2.14,0.0,...,Mobile,Chicago Independents,41.878866,-87.625192,41.899602,-87.633308,32,LOOP,8,NEAR NORTH SIDE
3,3,058322b4ecd94483ab01630b6a07da6221fbac7f,c9867d006415cbc16529555f98cdeb44cb53aeaf1d9ae7...,2022-01-01 00:00:00,2022-01-01 00:00:00,0,0.00,3.25,0.00,0.0,...,Cash,Taxi Affiliation Services,41.857184,-87.620335,41.857184,-87.620335,33,NEAR SOUTH SIDE,33,NEAR SOUTH SIDE
4,4,0f0c856e620e6b4dfd2bb1e921d966dd179eeca1,b21050ab3ad3d0972fd6378f6bf4d0251a8a7af42e6e0e...,2022-01-01 00:00:00,2022-01-01 00:00:00,33,0.17,63.27,0.00,0.0,...,Cash,Flash Cab,41.965812,-87.655879,41.965812,-87.655879,3,UPTOWN,3,UPTOWN


In [5]:
# Explore dataset

taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265374 entries, 0 to 265373
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   taxi_trip_id                265374 non-null  int64  
 1   trip_id                     265374 non-null  object 
 2   taxi_id                     265374 non-null  object 
 3   trip_start_timestamp        265374 non-null  object 
 4   trip_end_timestamp          265374 non-null  object 
 5   trip_seconds                265374 non-null  int64  
 6   trip_miles                  265374 non-null  float64
 7   fare                        265374 non-null  float64
 8   tips                        265374 non-null  float64
 9   tolls                       265374 non-null  float64
 10  extras                      265374 non-null  float64
 11  trip_total                  265374 non-null  float64
 12  payment_type                265374 non-null  object 
 13  company       

### Data Transformation

In [7]:
# Convert trip_start_timestamp & trip_end_timestamp column datatypes from object to datetime

taxi_data['trip_start_timestamp'] = pd.to_datetime(taxi_data['trip_start_timestamp'])
taxi_data['trip_end_timestamp'] = pd.to_datetime(taxi_data['trip_end_timestamp'])

In [8]:
# datetime_dim(start) table

datetime_dim = taxi_data[['trip_start_timestamp','trip_end_timestamp']].reset_index(drop=True)

datetime_dim['trip_start_timestamp'] = datetime_dim['trip_start_timestamp']
datetime_dim['start_hour'] = datetime_dim['trip_start_timestamp'].dt.hour
datetime_dim['start_day'] = datetime_dim['trip_start_timestamp'].dt.day
datetime_dim['start_month'] = datetime_dim['trip_start_timestamp'].dt.month
datetime_dim['start_year'] = datetime_dim['trip_start_timestamp'].dt.year
datetime_dim['start_weekday'] = datetime_dim['trip_start_timestamp'].dt.weekday

In [9]:
# datetime_dim(end) table

datetime_dim['trip_end_timestamp'] = datetime_dim['trip_end_timestamp']
datetime_dim['end_hour'] = datetime_dim['trip_end_timestamp'].dt.hour
datetime_dim['end_day'] = datetime_dim['trip_end_timestamp'].dt.day
datetime_dim['end_month'] = datetime_dim['trip_end_timestamp'].dt.month
datetime_dim['end_year'] = datetime_dim['trip_end_timestamp'].dt.year
datetime_dim['end_weekday'] = datetime_dim['trip_end_timestamp'].dt.weekday

In [10]:
# Create index for datetime_dim table

datetime_dim['datetime_id'] = datetime_dim.index


# Create datetime_dim tables

datetime_dim = datetime_dim[['datetime_id', 'trip_start_timestamp','start_hour', 'start_day', 
                             'start_month', 'start_year', 'start_weekday', 'trip_end_timestamp',
                            'end_hour', 'end_day', 'end_month', 'end_year', 'end_weekday']]

In [11]:
# Read the dataset

datetime_dim.head()

,datetime_id,trip_start_timestamp,start_hour,start_day,start_month,start_year,start_weekday,trip_end_timestamp,end_hour,end_day,end_month,end_year,end_weekday
0,0,2022-01-01,0,1,1,2022,5,2022-01-01 00:00:00,0,1,1,2022,5
1,1,2022-01-01,0,1,1,2022,5,2022-01-01 00:15:00,0,1,1,2022,5
2,2,2022-01-01,0,1,1,2022,5,2022-01-01 00:30:00,0,1,1,2022,5
3,3,2022-01-01,0,1,1,2022,5,2022-01-01 00:00:00,0,1,1,2022,5
4,4,2022-01-01,0,1,1,2022,5,2022-01-01 00:00:00,0,1,1,2022,5


In [12]:
# Create trip_distance_dim table

trip_distance_dim = taxi_data[['trip_miles']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index

# Conversion factor: 1 mile = 1.60934 kilometers
conversion_factor = 1.60934
trip_distance_dim['trip_km'] = taxi_data['trip_miles'] * conversion_factor

trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_miles', 'trip_km']]

In [13]:
# Read the dataset

trip_distance_dim.head()

,trip_distance_id,trip_miles,trip_km
0,0,4.83,7.773112
1,1,2.07,3.331334
2,2,2.48,3.991163
3,3,0.00,0.000000
4,4,0.17,0.273588


In [14]:
# Create trip_hour_dim table

trip_hour_dim = taxi_data[['trip_seconds']].reset_index(drop=True)
trip_hour_dim['trip_hour_id'] = trip_hour_dim.index
trip_hour_dim['trip_hour'] = taxi_data['trip_seconds'] / 3600
trip_hour_dim = trip_hour_dim[['trip_hour_id', 'trip_seconds', 'trip_hour']]

In [15]:
# Read the dataset

trip_hour_dim.head()

,trip_hour_id,trip_seconds,trip_hour
0,0,536,0.148889
1,1,897,0.249167
2,2,2200,0.611111
3,3,0,0.000000
4,4,33,0.009167


In [16]:
# Create payment_type_dim

payment_type_id = {
    'Cash':1,
    'Mobile':2, 
    'Credit Card':3, 
    'Unknown':4, 
    'Prcard':5, 
    'No Charge':6,
    'Dispute':7    
}

payment_type_dim = taxi_data[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_code'] = payment_type_dim['payment_type'].map(payment_type_id)
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type_code','payment_type']]

In [17]:
# Read the dataset

payment_type_dim.head()

,payment_type_id,payment_type_code,payment_type
0,0,1,Cash
1,1,1,Cash
2,2,2,Mobile
3,3,1,Cash
4,4,1,Cash


In [18]:
# Create company_dim table

company_ID = {
    'Globe Taxi':1, 
    'Sun Taxi':2, 
    'Chicago Independents':3,
    'Taxi Affiliation Services':4, 
    'Flash Cab':5,
    'Taxicab Insurance Agency, LLC':6, 
    'Choice Taxi Association':7,
    'City Service':8, 
    '24 Seven Taxi':9,
    'Medallion Leasin':10,
    'Patriot Taxi Dba Peace Taxi Associat':11, 
    'Top Cab Affiliation':12,
    'Blue Ribbon Taxi Association Inc.':13, 
    'Setare Inc':14,
    '312 Medallion Management Corp':15, 
    'U Taxicab':16
}


company_dim = taxi_data[['company']].reset_index(drop=True)
company_dim['company_id'] = company_dim.index
company_dim['company_code'] = company_dim['company'].map(company_ID)
company_dim['company_name'] = company_dim['company']
company_dim = company_dim[['company_id', 'company_code', 'company_name']]

In [19]:
# Read the dataset

company_dim.head()

,company_id,company_code,company_name
0,0,1.0,Globe Taxi
1,1,2.0,Sun Taxi
2,2,3.0,Chicago Independents
3,3,4.0,Taxi Affiliation Services
4,4,5.0,Flash Cab


In [20]:
# Create pickup_location_dim table

pickup_location_dim = taxi_data[['pickup_centroid_longitude', 'pickup_centroid_latitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_centroid_latitude','pickup_centroid_longitude']] 

In [21]:
# Read the dataset

pickup_location_dim.head()

,pickup_location_id,pickup_centroid_latitude,pickup_centroid_longitude
0,0,41.874005,-87.663518
1,1,41.899602,-87.633308
2,2,41.899602,-87.633308
3,3,41.857184,-87.620335
4,4,41.965812,-87.655879


In [22]:
# Create dropoff_location_dim table

dropoff_location_dim = taxi_data[['dropoff_centroid_longitude', 'dropoff_centroid_latitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_centroid_latitude','dropoff_centroid_longitude']]

In [23]:
# Read the dataset

dropoff_location_dim.head()

,dropoff_location_id,dropoff_centroid_latitude,dropoff_centroid_longitude
0,0,41.922761,-87.699155
1,1,41.878866,-87.625192
2,2,41.878866,-87.625192
3,3,41.857184,-87.620335
4,4,41.965812,-87.655879


In [25]:
# Create dropoff_comunity_area_dim table

dropoff_comunity_area_dim = taxi_data[['dropoff_communityID', 'dropoff_community_area']].reset_index(drop=True)
dropoff_comunity_area_dim['dropoff_community_area_id'] = dropoff_comunity_area_dim.index
dropoff_comunity_area_dim = dropoff_comunity_area_dim[['dropoff_community_area_id','dropoff_communityID', 'dropoff_community_area']]

In [26]:
# Read the data

dropoff_comunity_area_dim.head()

,dropoff_community_area_id,dropoff_communityID,dropoff_community_area
0,0,22,LOGAN SQUARE
1,1,32,LOOP
2,2,32,LOOP
3,3,33,NEAR SOUTH SIDE
4,4,3,UPTOWN


In [27]:
# Create pickup_comunity_area_dim table

pickup_comunity_area_dim = taxi_data[['pickup_communityID', 'pickup_community_area']].reset_index(drop=True)
pickup_comunity_area_dim['pickup_community_area_id'] = pickup_comunity_area_dim.index
pickup_comunity_area_dim = pickup_comunity_area_dim[['pickup_community_area_id','pickup_communityID', 'pickup_community_area']]

In [28]:
# Read the data

pickup_comunity_area_dim.head()

,pickup_community_area_id,pickup_communityID,pickup_community_area
0,0,28,NEAR WEST SIDE
1,1,8,NEAR NORTH SIDE
2,2,8,NEAR NORTH SIDE
3,3,33,NEAR SOUTH SIDE
4,4,3,UPTOWN


In [29]:
# Create taxi_trip_fact table

taxi_trip_fact = taxi_data.merge(datetime_dim, left_on='taxi_trip_id', right_on='datetime_id') \
            .merge(trip_distance_dim, left_on='taxi_trip_id', right_on='trip_distance_id') \
            .merge(payment_type_dim, left_on='taxi_trip_id', right_on='payment_type_id') \
            .merge(trip_hour_dim, left_on='taxi_trip_id', right_on='trip_hour_id') \
            .merge(company_dim, left_on='taxi_trip_id', right_on='company_id') \
            .merge(dropoff_location_dim, left_on='taxi_trip_id', right_on='dropoff_location_id') \
            .merge(pickup_location_dim, left_on='taxi_trip_id', right_on='pickup_location_id') \
            .merge(dropoff_comunity_area_dim, left_on='taxi_trip_id', right_on='dropoff_community_area_id') \
            .merge(pickup_comunity_area_dim, left_on='taxi_trip_id', right_on='pickup_community_area_id') \
            [['taxi_trip_id', 'trip_id', 'taxi_id', 'datetime_id', 'trip_distance_id', 'payment_type_id',
            'company_id', 'dropoff_community_area_id', 'dropoff_location_id', 'pickup_community_area_id',
            'pickup_location_id', 'trip_hour_id', 'fare', 'tips', 'tolls', 'extras', 'trip_total']]

In [30]:
# Read the data

taxi_trip_fact.head()

,taxi_trip_id,trip_id,taxi_id,datetime_id,trip_distance_id,payment_type_id,company_id,dropoff_community_area_id,dropoff_location_id,pickup_community_area_id,pickup_location_id,trip_hour_id,fare,tips,tolls,extras,trip_total
0,0,54d812a0b88f8f9707825261014b3563a0a60ace,f98ae5e71fdda8806710af321dce58002146886c013f41...,0,0,0,0,0,0,0,0,0,14.75,0.00,0.0,0.0,14.75
1,1,7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef,8eca35a570101ad24c638f1f43eecce9d0cb7843e13a75...,1,1,1,1,1,1,1,1,1,9.75,0.00,0.0,1.5,11.25
2,2,f1a650ee419b4e52d766432e6f86eca3457bfb37,e2d8418fcdb061eee0a4318fba0a6a1200aaff0143feb0...,2,2,2,2,2,2,2,2,2,9.36,2.14,0.0,0.0,11.50
3,3,058322b4ecd94483ab01630b6a07da6221fbac7f,c9867d006415cbc16529555f98cdeb44cb53aeaf1d9ae7...,3,3,3,3,3,3,3,3,3,3.25,0.00,0.0,0.0,3.25
4,4,0f0c856e620e6b4dfd2bb1e921d966dd179eeca1,b21050ab3ad3d0972fd6378f6bf4d0251a8a7af42e6e0e...,4,4,4,4,4,4,4,4,4,63.27,0.00,0.0,0.0,63.27


In [ ]:
#from google.cloud import storage

# Google Cloud Storage bucket and object name
#bucket_name = "your-bucket-name"
#object_name = "data2022.csv"

# Create a Google Cloud Storage client
#client = storage.Client()

# Get the bucket
#bucket = client.bucket(bucket_name)

# Upload the file to the bucket
#blob = bucket.blob(object_name)
#blob.upload_from_filename('data2022.csv')

#print("CSV file uploaded to Google Cloud Storage")